In [ ]:
# Install necessary libraries if you haven't already

In [ ]:
!pip install transformers datasets

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# Load the Llama3 model and tokenizer

In [ ]:
model_name = "Meta-Llama-3.1-8B"  # Replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Load and preprocess your dataset

In [ ]:
texts = pdf_to_text('../data/pdfs')  # Reuse function from data exploration

# Create a dataset for fine-tuning

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = Dataset.from_pandas(pd.DataFrame(texts))
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tuning configuration

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()

# Save the fine-tuned model


In [ ]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')
